In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
def load_data_label(feature_path, label_file_path, num_instance = 2000):
    y = pd.read_csv(label_file_path, header=0).drop('song_id',axis=1)
    # Load the data from each file and concatenate into a single feature matrix
    feature_matrix = None
    label_matrix = y[:num_instance]

    for i in range(1, num_instance + 1):
        file_path = os.path.join(feature_path, str(i) + ".mp3.npy")
        # print("Processed file " + file_path)
        data = np.load(file_path)
        data = [np.expand_dims(data, axis=-1)]
        if feature_matrix is None:
            feature_matrix = data
        else:
            feature_matrix = np.concatenate((feature_matrix, data), axis=0)
    return feature_matrix, label_matrix


In [4]:
def model_middle_feature_construction():
    model = keras.Sequential([
    # 1st Layer
    layers.Conv2D(64, kernel_size=(5, 5), strides = 2 ,activation='relu', padding="valid",input_shape=feature_matrix.shape[1:]),
    layers.BatchNormalization(),
    # 2nd Layer
    layers.Conv2D(64, kernel_size=(3, 3), strides = 1 ,activation='relu', padding="same"),
    layers.BatchNormalization(),
    # 3rd Layer
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.3),
    # 4th Layer
    layers.Conv2D(128, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # 5th Layer
    layers.Conv2D(128, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # 6th Layer
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.3),
    # 7th
    layers.Conv2D(256, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # # 8th
    layers.Conv2D(256, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # # 9th
    layers.Conv2D(384, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # # 10th 
    layers.Conv2D(512, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # # 11th
    layers.Conv2D(256, kernel_size=(3, 3), strides = 1, activation='relu', padding="same"),
    layers.BatchNormalization(),
    # # 12th
    layers.AveragePooling2D(),
    layers.Flatten(),
    layers.Dense(256),
    layers.Dense(7),
])
    return model

In [5]:
## File parameter
FOLDER_PATH = os.getcwd()
SPECTROGRAM_313x149_SAVE_DIR = FOLDER_PATH+ "/spectograms_313x149_Mel_func"
ANNOTATION_FILE_PATH = FOLDER_PATH + "/annotations.csv"

## DATA PARAMETER
NUM_INSTANCE = 200
TEST_SIZE = 0.2

## Model Parameter
BATCH_SIZE = 8
EPOCHS = 40
MODEL_SAVE = True

In [6]:
 # load feature, label
feature_matrix, label_matrix = load_data_label(SPECTROGRAM_313x149_SAVE_DIR, ANNOTATION_FILE_PATH, NUM_INSTANCE)

# split train, test
train_features, test_features, train_labels, test_labels = train_test_split(feature_matrix, label_matrix, test_size=TEST_SIZE, random_state=42)

In [7]:
model = model_middle_feature_construction()

opt = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='MSE', metrics=['mse', 'mae', 'mape', 'accuracy'])

In [8]:
history = model.fit(train_features, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/40
20/20 [==============================] - 47s 2s/step - loss: 192.5058 - mse: 192.5058 - mae: 10.1476 - mape: 198.1258 - accuracy: 0.1437
Epoch 2/40
20/20 [==============================] - 43s 2s/step - loss: 80.7084 - mse: 80.7084 - mae: 6.9441 - mape: 141.2895 - accuracy: 0.1813
Epoch 3/40
20/20 [==============================] - 44s 2s/step - loss: 47.2304 - mse: 47.2304 - mae: 5.3036 - mape: 110.3720 - accuracy: 0.2188
Epoch 4/40
20/20 [==============================] - 42s 2s/step - loss: 45.5931 - mse: 45.5931 - mae: 5.1987 - mape: 104.2619 - accuracy: 0.2062
Epoch 5/40
20/20 [==============================] - 42s 2s/step - loss: 28.8732 - mse: 28.8732 - mae: 4.1983 - mape: 86.2086 - accuracy: 0.1875
Epoch 6/40
20/20 [==============================] - 48s 2s/step - loss: 16.6161 - mse: 16.6161 - mae: 3.2588 - mape: 65.6443 - accuracy: 0.2188
Epoch 7/40
20/20 [==============================] - 42s 2s/step - loss: 12.9325 - mse: 12.9325 - mae: 2.8154 - mape: 57.2067 - ac

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).batch(32)

In [11]:
evauation = model.evaluate(dataset)

2/2 [==============================] - 3s 483ms/step - loss: 17.1878 - mse: 17.1878 - mae: 3.1619 - mape: 61.2895 - accuracy: 0.0500


In [12]:
if MODEL_SAVE == True:
        model.save(FOLDER_PATH+"/model/mel-model")

INFO:tensorflow:Assets written to: C:\Vu\Thesis\code\updated16.4/model/mel-model\assets


INFO:tensorflow:Assets written to: C:\Vu\Thesis\code\updated16.4/model/mel-model\assets
